In [1]:
import os, sys
import pandas as pd
import re
import numpy as np

In [2]:
def decode(x):
    f1 = float(re.search(pattern=r'f1:([\d].[\d]+)', string=x).groups()[0])
    all_f1 = x[x.find('[')+1:x.find(']')].replace('\n','')
    return f1, all_f1
def split_string(df):
    df = df.copy()
    select_AU = [ 'AU01','AU02', 'AU04', 'AU06', 'AU07', 'AU10', \
                 'AU12', 'AU14', 'AU15', 'AU17','AU23', 'AU24']
    for i, val in enumerate(select_AU):
        df[val] = df.all_f1.map(lambda x: float(re.split(pattern='[\ ]+', string=x.strip())[i]))
    del df['all_f1']
    df['f1'] = df['f1'].map(lambda x: float(x))
    return df
def summary_result_step1(content, num_epochs=10, num_times=3):
    index = []
    for i in range(num_times):
        for epoch in range(num_epochs):
            index.append('%02d_%02d'%(i,epoch))
    trn_df = pd.DataFrame(index=index, columns=['time', 'epoch', 'f1', 'all_f1'])
    val_clean_df = trn_df.copy()
    val_occ_df = trn_df.copy()
    test_clean_df = trn_df.copy()
    test_occ_df = trn_df.copy()
    for i in range(num_times):
        start = content.find('time%02d'%i)
        end = content.find('time%02d'%(i+1))
        one_time = content[start:end]
        for epoch in range(num_epochs):
            start = one_time.find('epochs %02d' % epoch)
            end = one_time.find('epochs %02d' % (epoch+1))
            if end==-1:one_epoch = one_time[start:]
            else: one_epoch = one_time[start:end]

            trn = one_epoch[one_epoch.find('trn'):one_epoch.find('val_clean')]
            f1, all_f1 = decode(trn)
            trn_df.loc['%02d_%02d'%(i,epoch), 'time'] = i
            trn_df.loc['%02d_%02d'%(i,epoch), 'epoch'] = epoch
            trn_df.loc['%02d_%02d'%(i,epoch), 'f1'] = f1
            trn_df.loc['%02d_%02d'%(i,epoch), 'all_f1'] = all_f1

            val_clean = one_epoch[one_epoch.find('val_clean'):one_epoch.find('val_occ')]
            f1, all_f1 = decode(val_clean)
            val_clean_df.loc['%02d_%02d'%(i,epoch), 'time'] = i
            val_clean_df.loc['%02d_%02d'%(i,epoch), 'epoch'] = epoch
            val_clean_df.loc['%02d_%02d'%(i,epoch), 'f1'] = f1
            val_clean_df.loc['%02d_%02d'%(i,epoch), 'all_f1'] = all_f1

            val_occ = one_epoch[one_epoch.find('val_occ'):one_epoch.rfind(']')+1]
            f1, all_f1 = decode(val_occ)
            val_occ_df.loc['%02d_%02d'%(i,epoch), 'time'] = i
            val_occ_df.loc['%02d_%02d'%(i,epoch), 'epoch'] = epoch
            val_occ_df.loc['%02d_%02d'%(i,epoch), 'f1'] = f1
            val_occ_df.loc['%02d_%02d'%(i,epoch), 'all_f1'] = all_f1
    trn_df = split_string(trn_df)
    val_clean_df = split_string(val_clean_df)
    val_occ_df = split_string(val_occ_df)
    return trn_df, val_clean_df, val_occ_df

def summary_result_step2(trn_df, val_clean_df, val_occ_df, num_times=3, num_epochs=10):
    """
        获得每一轮的平均结果
    """
    select_AU = [ 'AU01','AU02', 'AU04', 'AU06', 'AU07', 'AU10', \
                 'AU12', 'AU14', 'AU15', 'AU17','AU23', 'AU24']
    sel_val_clean = pd.DataFrame(index=list(range(num_epochs)), columns=['epoch', 'f1', *select_AU])
    sel_val_occ = sel_val_clean.copy()
    for epoch in range(num_epochs):
        tmp = val_clean_df[val_clean_df.epoch==epoch].copy()
        sel_val_clean.loc[epoch] = tmp.mean(axis=0).loc[sel_val_clean.columns]
        tmp = val_occ_df[val_occ_df.epoch==epoch]
        sel_val_occ.loc[epoch] = tmp.mean(axis=0).loc[sel_val_occ.columns]
    return sel_val_clean, sel_val_occ
def summary_result_step3(sel_val_clean, sel_val_occ):
    summary_df = pd.DataFrame(index=['val_clean', 'val_occ'], columns=sel_val_clean.columns)
    summary_df.loc['val_clean'] = sel_val_clean.loc['mean']
    summary_df.loc['val_occ'] = sel_val_occ.loc['mean']
    return summary_df

# PACNN

In [5]:
log_path = './log.txt'
with open(log_path, 'r') as f:
    content = f.read()
trn_df, val_clean_df, val_occ_df = summary_result_step1(content, num_epochs=10, num_times=3)
sel_val_clean, sel_val_occ = summary_result_step2(trn_df, val_clean_df, val_occ_df, num_epochs=10, num_times=3)
sel_val_clean.to_csv(log_path[:-4]+'_clean.csv', index=False)
sel_val_occ.to_csv(log_path[:-4]+'_occ.csv', index=False)
index = sel_val_clean.f1.astype(np.float32).idxmax()
tmp = pd.DataFrame()
tmp = tmp.append(sel_val_clean.iloc[index], ignore_index=True)
tmp = tmp.append(sel_val_occ.iloc[index], ignore_index=True)
tmp.rename(axis=0, mapper={0: 'clean', 1:'occ'}, inplace=True)
tmp